# Import des modules

In [13]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
import json

# Lecture des données

In [33]:
db = pd.read_csv("../Ars_data_v_2/output_data.csv", encoding="utf-8", low_memory=False)
db.drop(db.columns[0], axis=1, inplace=True)

# Features engineering

In [34]:
code_dep = set(db['code_dep'])
db['PGP19_>60'].fillna(0, inplace=True)
for code in code_dep:
    db_temp = db[db['code_dep'] == code]
    sum_pop = db_temp['PM16'].sum()
    db.loc[(db['code_dep'] == code),'PPMsD16']= db['PM16']/sum_pop
    db.loc[(db['code_dep'] == code),'PD16']=sum_pop
db=db.where(pd.notnull(db), None)
db=db.dropna()
db['PM16']=db['PM16'].astype(int)
db['PD16']=db['PD16'].astype(int)
db['code_region']=db['code_region'].astype(int)

# Séparation des données

In [36]:
var_desc = db[["Nom_com","code_dep","code_commune_INSEE",
               "nom_departement","nom_region","code_postal","code_region","PM16"]] 
var_num = db[["RFM16","TN16","PPMsD16","NSP19","DP16","PGP19","PGP19_>60",
              "PC16_24_64","PC16_15_24","PPop16_<18","PPop16_>65","PRP16"]]
X = var_num

Second tableau pour les données à afficher

In [37]:
# Retrait des variables ou il n'y a pas besoin de re multiplier par la population 
db_display_value = var_num
l = list(db_display_value.columns)
for elem in ["DP16","TN16","RFM16","PGP19_>60"]:
        l.remove(elem)
        
# Re calcule des valeurs initiale et ajout au denominateur de la population municipale et/ou de la population departementale      
for name in l:
    if name != "PPMsD16":
        db_display_value.loc[:, name] = db_display_value[name]*db['PM16']
        db_display_value[name]=db_display_value[name].astype(int)
        db_display_value.loc[:, name] = db_display_value[name].astype(str) + "/" + db['PM16'].astype(str)
    else:
        db_display_value.loc[:,name] = db_display_value[name]*db['PD16']
        db_display_value[name]=db_display_value[name].astype(int)
        db_display_value.loc[:, name] = db_display_value[name].astype(str) + "/" + db['PD16'].astype(str)


C:\Users\joaka\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\joaka\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\joaka\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Ajout des données informative à ce second tableau

In [39]:
db_final = pd.concat([var_desc,db_display_value], axis=1) 

# Calcule des seuils à afficher 

In [5]:
# median_sup = var_num.median()
# std = var_num.std()
# first_q = var_num.quantile(0.25)
# third_q = var_num.quantile(0.75)

In [6]:
# def calcule_indicateur(df):
#     i = 0
#     colnames = df.columns
#     for colname in colnames:
#         df[colname] = df[colname].apply(lambda x: 1 if(x>median[i] + std[i]) else 
#                                         (-1 if(x<median[i] - std[i]) else 0))
#         i+=1
#     return df

In [7]:
# display_value = calcule_indicateur(X)

In [8]:
# col_sum = display_value.sum(axis=1)

In [9]:
# final = pd.concat([var_desc,display_value],axis=1)
# final['Score'] = col_sum/len(var_num.columns)

Pour Théo (Auto Complétion)

In [10]:
# final['Nom_com'].to_csv("liste_des_communes", sep=",")

# Export en json

In [75]:
d_display = db_final.set_index("Nom_com").T.to_dict()
d_value = db.set_index("Nom_com").T.to_dict()

C:\Users\joaka\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.
C:\Users\joaka\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


In [12]:
# with open('data.json', 'w', encoding='utf8') as file:
#      file.write(json.dumps(d, ensure_ascii=False))

# Classification (Kmeans)

In [13]:
# kmeans = KMeans(n_clusters=12,init='k-means++', n_init=10, random_state=0)

In [14]:
# kmeans.fit(X)

In [15]:
# wcss = []
# for i in range(1, 50):
#     kmeans = KMeans(n_clusters=i,init='k-means++', n_init=10, random_state=0)
#     kmeans.fit(X)
#     wcss.append(kmeans.inertia_)
# plt.plot(range(1, 30), wcss)
# plt.title('Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()

# Plus proche voisins (KDtree) 

In [16]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='kd_tree').fit(X)
distances, indices = nbrs.kneighbors(X)
indices

array([[    0, 20752, 19981, 17452, 14640,   336],
       [    1,  5778, 17396,  2491, 28474, 28902],
       [    2,  4786, 17513, 21510, 16863, 21083],
       ...,
       [29356, 17915, 22615, 18687, 20182, 22654],
       [29357,   151, 25707,  5538, 13833, 23545],
       [29358, 17184,  9424,  6930, 25597,  7136]], dtype=int64)

In [17]:
proche_voisin = []
for i in range(len(db.index)):
    proche_voisin.append(list(db.iloc[indices[i][1:]]['Nom_com']))

In [18]:
db_display_value['Proche_voisin'] = proche_voisin

# Exports des données 

Export des seuils pour clément

1ere version 

In [20]:
# d = {}
# for name in X.columns:
#     median = X[name].median()
#     sd = X[name].std()
#     d[name] = [{'jaune':X[name].median,'milieu':X[name].quantile(0.5),'haut':X[name].quantile(0.75),'sd':sd}]

2e version

In [69]:
db['PRP16'].quantile(0.5)

0.005681818181818182

In [70]:
d_seuil = {"RFM16":19554.0,"TN16":8.0,"PPMsD16":0.0009156237049825108,"NSP19":0,"DP16":61.7,"PGP19":0,"PGP19_>60":0,
              "PC16_24_64":0.0473,"PC16_15_24":0.00986,"PPop16_<18":0.222,"PPop16_>65":0.1964,"PRP16":0.0056}

In [72]:
with open('seuils.json', 'w', encoding='utf8') as file:
     file.write(json.dumps(d_seuil, ensure_ascii=False))

Export du jeu de données final

In [76]:
with open('data.json', 'w', encoding='utf8') as file:
     file.write(json.dumps(d_value, ensure_ascii=False))

In [77]:
with open('data_display.json', 'w', encoding='utf8') as file:
     file.write(json.dumps(d_display, ensure_ascii=False))